# PROBLEMA 3: **Cáncer de Seno**  

### Entrega: Daniel Rojo Mata

## **Enunciado del problema**

Utiliza el conjunto de datos de cáncer de seno de Wisconsin, que contiene 699 registros de tumores, de los cuales 458 son benignos y 241 son malignos. Cada registro consta de los siguientes atributos:

### **1. Preparación de los datos:**
   - Divide aleatoriamente el conjunto de datos en un subconjunto de entrenamiento con el 60 % de los datos, un subconjunto de validación con el 20 % y un subconjunto de prueba con el 20 % restante, utilizando 0 como semilla para tu generador de números aleatorios.

### **2. Entrenamiento del clasificador:**
   - Entrena distintos clasificadores para los tumores de seno y evalúalos tanto con el subconjunto de entrenamiento como con el subconjunto de validación, discutiendo su desempeño.

### **3. Manejo de datos faltantes:**
   - Investiga estrategias para rellenar los datos faltantes, ya que existen 16 registros en el conjunto de datos con un atributo no especificado. Utiliza las estrategias que consideres más adecuadas y discute el impacto en el desempeño del clasificador.

### **4. Evaluación del modelo:**
   - Reporta el porcentaje de predicciones correctas en el subconjunto de prueba para el clasificador que tenga el mejor rendimiento en el subconjunto de validación.

## **Descripción del conjunto de datos**
El conjunto de datos de cáncer de seno de Wisconsin incluye registros de tumores con las siguientes características:
- **Atributos:** Contiene varias mediciones y características relacionadas con los tumores.
- **Etiquetas:** Indica si el tumor es benigno o maligno.


In [55]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

## Se procesa la data

In [2]:
archivo = "cancer.csv" # Nombre del archivo, es importante que esté en la misma carpeta que este archivo
data = pd.read_csv(archivo, # Se importa el archivo
                  header=None, # Quitamos el encabezado
                  sep=',' # Separador
                  )

## Observación: Sobre el procesamiento de la data 
Los datos están organizados de la siguiente manera:

In [93]:
data.head(5) # Ver los primeros 5 elementos de la data

,0
0,"1000025,5,1,1,1,2,1,3,1,1,2"
1,"1002945,5,4,4,5,7,10,3,2,1,2"
2,"1015425,3,1,1,1,2,2,3,1,1,2"
3,"1016277,6,8,8,1,3,4,3,7,1,2"
4,"1017023,4,1,1,3,2,1,3,1,1,2"


Tomando el primer elemento se obtiene: 

In [92]:
data.iloc[0,:] # Ver el primer renglón

0    1000025,5,1,1,1,2,1,3,1,1,2
Name: 0, dtype: object

Por lo que no es posible procesar de una manera adecuada los datos. 
Es por eso que se usa la función **procesar_data** la cual retorna 
los datos como una lista de listas, en donde cada lista contiene elementos de tipo entero.

In [90]:
def procesar_data(data):
    """
    Función que retorna una lista de listas con valores enteros, 
    convirtiendo los datos del DataFrame en una estructura manejable.

    Parámetros: 
        data: DataFrame de pandas con los datos en formato de una sola columna con valores separados por comas.

    Return: 
        lista_numerica: lista de listas con valores enteros, manteniendo los "?" como están.
    """
    
    lista_inicial = data.values.tolist()  # Convierte el DataFrame en una lista de listas (cada fila como lista).
    lista_numerica = []  # Lista que almacenará los datos convertidos.

    for lista in lista_inicial:  # Itera sobre cada fila del dataset.
        for i in range(len(lista)):  
            lista_split = lista[i].split(",")  # Divide la cadena en una lista separando por comas.
            lista_auxiliar = []  # Lista temporal para almacenar los valores convertidos.

            for k in lista_split:  # Itera sobre los valores de la lista dividida.
                if k != "?":  # Si el valor no es un "?", lo convierte en entero.
                    lista_auxiliar.append(int(k))
                else:  # Si es "?", lo mantiene como está para su posterior tratamiento.
                    lista_auxiliar.append(k)

            lista_numerica.append(lista_auxiliar)  # Agrega la lista procesada a la lista principal.

    return lista_numerica  # Retorna la lista de listas con valores convertidos.

# Lista inicial

Lista que se usará de aquí en adelante.

In [94]:
lista_inicial = procesar_data(data) # Se procesa la data con la función anterior

# Datos faltantes

Se tienen 16 registros en los cuales no se tiene completa la información.
La función **listas_con_interrogacion** devuelve estas listas (registros).

In [95]:
def listas_con_interrogacion(lista_de_listas):
    """
    Función que devuelve las listas que incluyen el carácter "?".
    
    Parámetros: 
        lista_de_listas: lista cuyos elementos son listas con valores numéricos
        o con el carácter "?".
    
    Return:
        Una lista que contiene solo las sublistas que contienen el carácter "?".
    """
    # Utiliza una comprensión de listas para filtrar las sublistas que contienen "?".
    return [lista for lista in lista_de_listas if "?" in lista]

In [96]:
# Listas en donde se tiene un elemento con "?"
listas_interrogacion = listas_con_interrogacion(lista_inicial) 
# Son 16 cosos
listas_interrogacion

[[1057013, 8, 4, 5, 1, 2, '?', 7, 3, 1, 4],
 [1096800, 6, 6, 6, 9, 6, '?', 7, 8, 1, 2],
 [1183246, 1, 1, 1, 1, 1, '?', 2, 1, 1, 2],
 [1184840, 1, 1, 3, 1, 2, '?', 2, 1, 1, 2],
 [1193683, 1, 1, 2, 1, 3, '?', 1, 1, 1, 2],
 [1197510, 5, 1, 1, 1, 2, '?', 3, 1, 1, 2],
 [1241232, 3, 1, 4, 1, 2, '?', 3, 1, 1, 2],
 [169356, 3, 1, 1, 1, 2, '?', 3, 1, 1, 2],
 [432809, 3, 1, 3, 1, 2, '?', 2, 1, 1, 2],
 [563649, 8, 8, 8, 1, 2, '?', 6, 10, 1, 4],
 [606140, 1, 1, 1, 1, 2, '?', 2, 1, 1, 2],
 [61634, 5, 4, 3, 1, 2, '?', 2, 3, 1, 2],
 [704168, 4, 6, 5, 6, 7, '?', 4, 9, 1, 2],
 [733639, 3, 1, 1, 1, 2, '?', 3, 1, 1, 2],
 [1238464, 1, 1, 1, 1, 1, '?', 2, 1, 1, 2],
 [1057067, 1, 1, 1, 1, 1, '?', 1, 1, 1, 2]]

Por lo que antes de dividir el conjunto de datos en las proporciones que se piden,
se piensan las siguientes estrategias para "completar" esos datos.

# **Estrategias para llenar los datos faltantes**

En este trabajo se emplean dos estrategias para manejar los valores faltantes representados por `"?"`:  

## **1. Eliminar filas con `?`**  
Esta estrategia consiste en eliminar directamente todas las filas que contienen valores desconocidos.  

✅ **Ventajas:**  
- Simple de implementar.  
- Evita cualquier sesgo introducido por valores imputados.  

❌ **Desventajas:**  
- Se pierde información útil si hay muchas filas afectadas.  
- Puede reducir el tamaño del conjunto de datos, lo que impacta la calidad del modelo.  

## **2. Imputación con la mediana**  
En este enfoque, los valores faltantes se reemplazan con la mediana de la columna correspondiente. La mediana es robusta a valores extremos, lo que la hace adecuada para datos sesgados.  

✅ **Ventajas:**  
- Conserva la mayor cantidad posible de datos.  
- Reduce el impacto de valores atípicos en comparación con la media.  

❌ **Desventajas:**  
- Puede introducir sesgo si los datos faltantes no están distribuidos aleatoriamente.  
- No captura variabilidad en los valores reemplazados.  

## Estrategia 1: Eliminar filas con "?"

Se borran estos datos en **lista_inicial**.

In [97]:
# Filtra las sublistas de lista_inicial para eliminar aquellas que contienen el carácter "?".
data_numpy_filtrado = [sublista for sublista in lista_inicial if sublista not in listas_con_interrogacion(lista_inicial)]

In [98]:
# Se observa que las lingitudes coinciden
len(data_numpy_filtrado) == len(lista_inicial)-len(listas_interrogacion)

True

In [99]:
# 699 - 16 = 683
len(data_numpy_filtrado)

683

### División del conjunto de datos utilizando las filas eliminadas con "?"

Para dividir la data, se usan los datos en **data_numpy_filtrado**

In [100]:
# IMPORTANTE: hacemos que data_numpy_filtrado sea un array para poder usar [:,:]
X_interrogacion = np.array(data_numpy_filtrado)[:, :-1]  # Todas las columnas excepto la última
y_interrogacion = np.array(data_numpy_filtrado)[:, -1]   # Solo la última columna

# Dividir en conjunto de entrenamiento (60%) y el resto (40%)
X_ent, X_temp, y_ent, y_temp = train_test_split(X_interrogacion, y_interrogacion, test_size=0.4, random_state=0)

# Dividir el conjunto temporal en validación (20%) y prueba (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

# Ver las dimensiones de los conjuntos anteriores
print(f'Tamaño del conjunto de entrenamiento: {X_ent.shape}')
print(f'Tamaño del conjunto de validación: {X_val.shape}')
print(f'Tamaño del conjunto de prueba: {X_test.shape}')

Tamaño del conjunto de entrenamiento: (409, 10)
Tamaño del conjunto de validación: (137, 10)
Tamaño del conjunto de prueba: (137, 10)


### Clasificador Gaussiano importado de Sklearn

In [101]:
# Crear una instancia del clasificador Naive Bayes Gaussiano
nb_classifier = GaussianNB()

# Entrenar el modelo con los datos de entrenamiento
nb_classifier.fit(X_ent, y_ent)

# Predecir para los conjuntos de entrenamiento, validación y prueba
predicciones_ent = nb_classifier.predict(X_ent)
predicciones_val = nb_classifier.predict(X_val)
predicciones_test = nb_classifier.predict(X_test)

# Calcular la precisión (accuracy)
accuracy_ent_interrogacion = accuracy_score(y_ent, predicciones_ent) * 100
accuracy_val_interrogacion = accuracy_score(y_val, predicciones_val) * 100
accuracy_test_interrogacion = accuracy_score(y_test, predicciones_test) * 100

# Imprimir los resultados
print(f"Precisión en entrenamiento: {accuracy_ent_interrogacion:.2f}%")
print(f"Precisión en validación: {accuracy_val_interrogacion:.2f}%")
print(f"Precisión en prueba: {accuracy_test_interrogacion:.2f}%")

Precisión en entrenamiento: 66.75%
Precisión en validación: 70.80%
Precisión en prueba: 70.80%


## Estrategia 2: imputación con la mediana



In [102]:
# Se convierte la lista_inicial en un array de numpy
lista_inicial_array = np.array(lista_inicial, dtype=object)  # dtype=object porque hay strings

# Reemplazar "?" con NaN
lista_inicial_array[lista_inicial_array == "?"] = np.nan

# Convertir a tipo numérico
lista_inicial_array = lista_inicial_array.astype(float)

# Calcular la mediana ignorando NaN
medianas = np.nanmedian(lista_inicial_array, axis=0)

# Reemplazar los NaN con la mediana de la columna correspondiente
indices_nan = np.where(np.isnan(lista_inicial_array))
lista_inicial_array[indices_nan] = np.take(medianas, indices_nan[1])

### División del conjunto de datos utilizando imputación con la mediana

Para dividir la data, se usan los datos en **data_numpy_mediana**

In [103]:
# Convertimos lista_inicial_array a un array NumPy si aún no lo es
data_numpy_mediana = np.array(lista_inicial_array)  

# Separa características (X) y etiquetas (y)
X_mediana = data_numpy_mediana[:, :-1]  # Todas las columnas excepto la última
y_mediana = data_numpy_mediana[:, -1]   # Solo la última columna

# Convertir etiquetas a enteros por si están como flotantes
y_mediana = y_mediana.astype(int)

# Dividir en conjunto de entrenamiento (60%) y el resto (40%)
X_ent, X_temp, y_ent, y_temp = train_test_split(X_mediana, y_mediana, test_size=0.4, random_state=0)

# Dividir el conjunto temporal en validación (20%) y prueba (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

# Ver las dimensiones de los conjuntos
print(f'Tamaño del conjunto de entrenamiento: {X_ent.shape}')
print(f'Tamaño del conjunto de validación: {X_val.shape}')
print(f'Tamaño del conjunto de prueba: {X_test.shape}')

Tamaño del conjunto de entrenamiento: (419, 10)
Tamaño del conjunto de validación: (140, 10)
Tamaño del conjunto de prueba: (140, 10)


### Clasificador Gaussiano importado de Sklearn

In [104]:
# Crear una instancia del clasificador Naive Bayes Gaussiano
nb_classifier = GaussianNB()

# Entrenar el modelo con los datos de entrenamiento
nb_classifier.fit(X_ent, y_ent)

# Predecir para los conjuntos de entrenamiento, validación y prueba
predicciones_ent = nb_classifier.predict(X_ent)
predicciones_val = nb_classifier.predict(X_val)
predicciones_test = nb_classifier.predict(X_test)

# Calcular la precisión (accuracy)
accuracy_ent_mediana = accuracy_score(y_ent, predicciones_ent) * 100
accuracy_val_mediana = accuracy_score(y_val, predicciones_val) * 100
accuracy_test_mediana = accuracy_score(y_test, predicciones_test) * 100

# Imprimir los resultados
print(f"Precisión en entrenamiento: {accuracy_ent_mediana:.2f}%")
print(f"Precisión en validación: {accuracy_val_mediana:.2f}%")
print(f"Precisión en prueba: {accuracy_test_mediana:.2f}%")

Precisión en entrenamiento: 82.82%
Precisión en validación: 80.00%
Precisión en prueba: 89.29%


## Comparación de las estrategias

In [105]:
# Analizar el desempeño
print("\nAnálisis del desempeño eliminando la data con '?':")
print("-" * 50)
print(f"La precisión en entrenamiento es {accuracy_ent_interrogacion:.2f}")
print(f"La precisión en validación es {accuracy_val_interrogacion:.2f}")
print(f"La precisión en prueba es {accuracy_test_interrogacion:.2f}")
print("\n")
print("\nAnálisis del desempeño utilizando la mediana:")
print("-" * 50)
print(f"La precisión en entrenamiento es {accuracy_ent_mediana:.2f}")
print(f"La precisión en validación es {accuracy_val_mediana:.2f}")
print(f"La precisión en prueba es {accuracy_test_mediana:.2f}")


Análisis del desempeño eliminando la data con '?':
--------------------------------------------------
La precisión en entrenamiento es 66.75
La precisión en validación es 70.80
La precisión en prueba es 70.80



Análisis del desempeño utilizando la mediana:
--------------------------------------------------
La precisión en entrenamiento es 82.82
La precisión en validación es 80.00
La precisión en prueba es 89.29
